In [10]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" #! specify gpu here
import sys
import numpy as np
import cv2
import time
import torch
import clip
from PIL import Image
from rich import print
from tqdm import tqdm
from pathlib import Path
import csv
import json
import click

from vision_pipeline.llm_data_generator.labelme_importer import LabelMeImporter


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
qa_disassembly_step = {
    "Q": "What is the next disassembly step?",
    "A": "" #  "Turn the device over to show the back."
}
qa_relations = {
    "Q": "What are the object relations?",
    "A": "" # "PCB inside HCA, ..."
}
qa_positions = {
    "Q": "What are the object positions?",
    "A": "" # "HCA at (302, 300), PCB at (...), ..."
}
qa_json_disassembly_step = {
    "Q": "What is the JSON formatted disassembly step?",
    "A": {
        "tool": "",
        "tool_args": "",
        "description": ""
    }
}

qas = [qa_disassembly_step, qa_relations, qa_positions, qa_json_disassembly_step]

In [12]:
kg_path = Path("~/datasets2/reconcycle/knowledge_graph").expanduser()
kg_nodes_path = kg_path / "data" / "nodes"
kg_edges_path = kg_path / "data" / "graph_edges.csv"

print("kg_nodes_path", kg_nodes_path)


kg_nodes_path /root/datasets2/reconcycle/knowledge_graph/data/nodes

In [13]:
hcas = []
with open(kg_edges_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if len(row) >= 2 and row[1] == 'hca':
            hcas.append(row[0])

print(hcas)

['kalo', 'qundis']

In [14]:
folder = kg_nodes_path / "kalo" / "disassembly"

labelme_importer = LabelMeImporter()


# todo: get graph relations

# todo: get positions of objects 

from IPython.utils import io 

with io.capture_output() as captured:
    img_paths, all_detections, all_graph_relations, modules, cameras = labelme_importer.process_labelme_dir(folder)




In [15]:
for img_path, detections, graph_relations, module, camera in zip(img_paths, all_detections, all_graph_relations, modules, cameras):

    stem = Path(img_path).stem
    qa_path = Path(img_path).parent / Path(str(stem) + "_qa.json")
    
    print("qa_path", qa_path)

    print("module", module.name)

    A_relations = graph_relations.to_text()
    print("A_relations:", A_relations)

    # include the module name in the relations
    if module is not None:
        A_relations = f"Module: {module.name}. " + A_relations
    
    positions_list = []
    for detection in detections:
        positions_list.append(f"{detection.label.name} at ({detection.center_px[0]}, {detection.center_px[1]})")
        # print("detection name", detection.label.name)
        # print(f"detection center: {detection.center_px[0]}, {detection.center_px[1]}")
    
    A_positions = ", ".join(positions_list) + ". "

    print("A_positions:", A_positions)
    
    # create or edit the <number>_qa.json file
    if qa_path.is_file():
        print("[red]qa already exists!")
        
        with open(qa_path, 'r+') as f:
            data = json.load(f)
            data_changed = False
            for qa in data:
                if qa["Q"] == qa_relations["Q"] and qa["A"] != A_relations:
                    if click.confirm(f"overwrite QA relations\ncurrent: {qa['A']}\nnew: {A_relations}?\n", default=True):
                        qa["A"] = A_relations
                        data_changed = True
                elif qa["Q"] == qa_positions["Q"] and qa["A"] != A_positions:
                    if click.confirm(f"overwrite QA positions\ncurrent: {qa['A']}\nnew: {A_positions}?\n", default=True):
                        qa["A"] = A_positions
                        data_changed = True

            if data_changed:
                f.seek(0)        # <--- should reset file position to the beginning.
                json.dump(data, f, indent=4)
                f.truncate()     # remove remaining part

    else:
        qa_disassembly_step_copy = qa_disassembly_step.copy()
        qa_relations_copy = qa_relations.copy()
        qa_positions_copy = qa_positions.copy()
        qa_json_disassembly_step_copy = qa_json_disassembly_step.copy()

        qa_relations_copy["A"] = A_relations
        qa_positions_copy["A"] = A_positions

        QAs_copy = [qa_disassembly_step_copy, qa_relations_copy, qa_positions_copy, qa_json_disassembly_step_copy]

        with open(qa_path, 'w', encoding='utf-8') as f:
            json.dump(QAs_copy, f, ensure_ascii=False, indent=4)



    

    




qa_path /root/datasets2/reconcycle/knowledge_graph/data/nodes/kalo/disassembly/1_qa.json

module vision

A_relations: Loose component: hca_front

A_positions: hca_front at (427, 428).

qa already exists!

...

qa_path /root/datasets2/reconcycle/knowledge_graph/data/nodes/kalo/disassembly/6_qa.json

module cutter

A_relations: Loose components: battery, internals, pcb_covered, pcb

A_positions: battery at (661, 595), internals at (836, 693), pcb_covered at (965, 299), pcb at (609, 754).

qa already exists!

In [16]:

# todo: create QAs